In [1]:
import os
os.listdir()

['convolve3.c',
 'Cython_testing.ipynb',
 'primes_python_compiled.py',
 'primes.pyx',
 'helloworld.pyx',
 'primes_python_compiled.c',
 'primes_python.py',
 'convolve1.c',
 'setup.py',
 'convolve2.pyx',
 'convolve3.pyx',
 'convolve1.pyx',
 '.ipynb_checkpoints',
 'primes.c',
 'helloworld.c',
 'ccg_py.c',
 'convolve2.c',
 'convolve_py.py']

This is in a file `setup.py` in the folder above along with `helloworld.pyx` which has one line: `print("Hello World")`

Type this in the terminal to "compile" it:  
`>>> python setup.py build_ext --inplace`

In [ ]:
import helloworld

Note that the cython build is specific to your python environment! Can't mix and match cython versions

This also works but you need to somehow set the language level appropriately

In [ ]:
import pyximport; pyximport.install()
import helloworld

## Start working with NumPy and working directly in notebook here

In [ ]:
%load_ext Cython

In [ ]:
%%cython --annotate

cdef int a = 0
for i in range(10):
    a += i
print(a)

In [ ]:
def compute_np(array_1, array_2, a, b, c):
    return np.clip(array_1, 2, 10)*a + array_2*b + c

In [ ]:
%%cython

import numpy as np


def clip(a, min_value, max_value):
    return min(max(a, min_value), max_value)


def compute(array_1, array_2, a, b, c):
    """
    This function must implement the formula
    np.clip(array_1, 2, 10) * a + array_2 * b + c

    array_1 and array_2 are 2D.
    """
    x_max = array_1.shape[0]
    y_max = array_1.shape[1]

    assert array_1.shape == array_2.shape

    result = np.zeros((x_max, y_max), dtype=array_1.dtype)

    for x in range(x_max):
        for y in range(y_max):
            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result[x, y] = tmp + c

    return result

In [ ]:
array_1 = np.random.uniform(0, 1000, size=(3000,2000)).astype(np.intc)
array_2 = np.random.uniform(0, 1000, size=(3000,2000)).astype(np.intc)
a, b, c = 4, 3, 9
%timeit compute_np(array_1, array_2, a, b, c)

In [ ]:
%timeit compute(array_1, array_2, a, b, c)

In [ ]:
%%cython

import numpy as np

# We now need to fix a datatype for our arrays. I've used the variable
# DTYPE for this, which is assigned to the usual NumPy runtime
# type info object.
DTYPE = np.intc

# cdef means here that this function is a plain C function (so faster).
# To get all the benefits, we type the arguments and the return value.
cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


def compute_typed(array_1, array_2, int a, int b, int c):

    # The "cdef" keyword is also used within functions to type variables. It
    # can only be used at the top indentation level (there are non-trivial
    # problems with allowing them in other places, though we'd love to see
    # good and thought out proposals for it).
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    assert array_1.shape == array_2.shape
    assert array_1.dtype == DTYPE
    assert array_2.dtype == DTYPE

    result = np.zeros((x_max, y_max), dtype=DTYPE)

    # It is very important to type ALL your variables. You do not get any
    # warnings if not, only much slower code (they are implicitly typed as
    # Python objects).
    # For the "tmp" variable, we want to use the same data type as is
    # stored in the array, so we use int because it correspond to np.intc.
    # NB! An important side-effect of this is that if "tmp" overflows its
    # datatype size, it will simply wrap around like in C, rather than raise
    # an error like in Python.

    cdef int tmp

    # Py_ssize_t is the proper C type for Python array indices.
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result[x, y] = tmp + c

    return result


In [ ]:
%timeit compute_typed(array_1, array_2, a, b, c)

In [ ]:
%%cython

import numpy as np

DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


def compute_memview(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):

    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    # array_1.shape is now a C array, no it's not possible
    # to compare it simply by using == without a for-loop.
    # To be able to compare it to array_2.shape easily,
    # we convert them both to Python tuples.
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [ ]:
%timeit compute_memview(array_1, array_2, a, b, c)